<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Streaming Multi Chan From File

### Description:
This script comes to demonstrate working with streaming mode with multi channels from file on a Proteus device.


### set up:
Use Proteues device and connect channel 1 and 3 to  the scoop.
This scripts works with PXI only because we need connection

### Proteus version: 
ALICE

# Streaming Mode From File

When working with streaming mode, the main application should continouosly push packets of waveform-data <br>
to instrument. The packets have constant size of 1 Mega ($1024\cdot1024$) bytes of waveform data.<br>

The operation of pushing packets to instrument must be very fast so the main application should run on the same<br>
computer that controls the hardware modules. Further more, the main application should load and run the DLL that<br> controls the hardware modules, and call the function that pushes a packet directly from the DLL.<br>

This means that the main application does not send SCPI commands over TCP to server that runs the hardware<br> controller, but passes the SCPI commands by calling appropriate function from the DLL.

Please note that if there is server running on the computer that controls the hardware, then it should be stop<br>
beacuse when it runs the DLL won't detect the hardware modules.

The DLL is called `TEProteus.dll`. It is a pure `c` DLL rather than *Dot Net* DLL (which might be slower), and<br>
it comes with header file `TEProteus.h` and lib file `TEProteus.lib` so it should be possible to load it from MATLB.<br>

In python one can use `teproteus` that loads the `TEProteus.dll` and gives access to its functions.<br>

In order to work in streaming mode

 1. Select channel by sending SCPI command: `:INSTrument:CHANnel[:SELect] <channel-number>`
 2. Acquire the stream-writing interface of the selected channel by calling the function<br>
    `TEProteusInst.acquire_stream_intf(chan_num)` from `teproteus`. 
 3. Start streaming mode in the selected channel by sending the SCPI command `:TRACe:STReaming ON`.
 4. Push packts of waveform data continouosly either with the function <br>
     - `TEProteusInst.push_stream_packet(self, stream_intf, bin_dat, bytes_offs, usec_wait)`  
     
    or with the lower level functions:<br>
     - `TEProteusInst.get_stream_empty_buff(self, stream_intf)`.
     - `TEProteusInst.put_stream_full_buff(self, stream_intf, full_buff, usec_wait)`.
     - `TEProteusInst.put_stream_empty_buff(self, stream_intf, empty_buff)`.  
    
    See their documentation in `teproteus_streaming.py`.
 5. Stop the streaming-mode at the end by sending the SCPI command `:TRACe:STReaming OFF`.


## Simple example
In this very simple example we load waveforms for each channel, each of them sould be at in the multiple size of one packet (1M samples), The packets of the selected waveforms (which we change every 30 seconds) are pushed (and downloaded) in a cyclic mode.
For multi channel streaming you can set the channels and the ptRepeat mode required.

In [ ]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus_streaming import TEProteusAdmin as TepAdmin
from teproteus_streaming import TEProteusInst as TepInst

In [ ]:
%matplotlib notebook
import numpy as np
import time
import random
from threading import Thread
import ipywidgets as widgets
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt

## Streaming & System Parameters

In [ ]:
############
channels = [1,3] # Select channels for streaming
sampMode = 8 # either 1,2,4,8. 0 # Please select Multi Sample mode.


inst = None
keep_looping = True
sclk = 2.5e9
slotIds = [8] # Please choose the correct slot.
sid = 8
lib_dir_path_ = None 
#lib_dir_path_= r'C:\Work_Svn\ProteusAwg_Adriana\x64\Release'
#lib_dir_path_= r'C:\Work_Svn\ProteusAwg_Alice\x64\Debug'




In [ ]:
print ("SCLK:{0}".format(sclk))
print ("Slot Id:{0}".format(slotIds))
print ("Sample Mode: {0}".format(sampMode))
print ("Number of Channels: {0}".format(len(channels)))



## Waveform Files

In [ ]:

DIR ='C:\\Users\\benny\\Documents\\ProteusFiles'
FILE1= DIR + '\\' + "wave_file_64.wav"
FILE2= DIR + '\\' + "wave_file_64_2.wav"
FILE3= DIR + '\\' + "wave_file_64_3.wav"
FILE4= DIR + '\\' + "wave_file_64_4.wav"

fileNames = [FILE1, FILE2, FILE3, FILE4]

for file in fileNames:
  print(file)


In [ ]:
pid = os.getpid()
print('process id {0}'.format(pid))


## Start Streaming Mode

In [ ]:
def start_streaming_mode(inst, sclk, channels):
    global sampMode
    
    for channel in channels:
        
        # Select channel
        rc = inst.send_scpi_cmd(':INST:CHAN {0}'.format(channel))

        # Turn output on
        rc = inst.send_scpi_cmd(':OUTP ON')

        # Send :TRACe:STReaming:MODE DYNamic
        rc = inst.send_scpi_cmd(':TRACe:STReaming:MODE DYNamic')
        
        # Set MultiSample Register Channels
        rc = inst.send_scpi_cmd(':PTRepeat X{0}'.format(sampMode))

    # Select sampling rate
    rc = inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sclk))

    # Turn streaming mode on

    # Send :TRACe:STReaming ON
    rc = inst.send_scpi_cmd(':TRAC:STR ON')

    time.sleep(0.1)

    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

    resp = inst.send_scpi_query(':TRAC:STR?')
    print('Streaming state: ' + resp)
    

## Push packets of waveform data continouosly from files

In [ ]:
def push_streaming_data(inst, channels, fileNames, num_packets, packet_size, dataType):
           
    #iwav1 = 0
    #wav1 = waves[iwav1]
    
    #iwav2 = 2
    #wav2 = waves[iwav2]
    dt = np.dtype(dataType)
    itemSize = dt.itemsize
    
    ipacket = 0
    byteOffs = 0

    # The maximal waiting-time when
    # pushing a full buffer (in microseconds)
    usWait = 50000
    
    global keep_looping

    t1 = time.time()    
    try:
        # Acquire the stream-writing interfaces of the actual channels:
        
        num_channels = len(channels)
        
        stream_interfaces = [ inst.acquire_stream_intf(channel) for channel in channels ]
        
        wav_indexes = [ ind % 3 for ind in range(num_channels) ]
        
        #wavs = [ waves[ind] for ind in wav_indexes ]
        
        #if not is_write_stream_active:
        #    raise RuntimeError('is_write_stream_active is False')
        waves = []
        # This is the stream-pushing loop  ..
        
        for ichan in range(num_channels):
          fname = fileNames[wav_indexes[ichan]]
          waves.append(np.fromfile(fname, dtype=dataType) )
            
        minWaveSize =  min([wav.shape[0] for wav in waves])
        num_packets = min(num_packets, minWaveSize*itemSize//packet_size)
        
        while keep_looping:
            byteOffs = ipacket * packet_size * itemSize
            wav = waves[wav_indexes[ichan]]
            for ichan in range(num_channels):
                stream_intf = stream_interfaces[ichan]

                # Push next packet (from the selected waveform)
                rc = inst.push_stream_packet(stream_intf, wav, byteOffs, usWait)
                if rc == -1:
                    print ("Push Packet Error {0}- Stopping".format(rc))
                    keep_looping = False
                    break 
                elif rc == 1:
                    print ("Push Packet Time Out Error {0}- Stopping".format(rc))
                    keep_looping = False
                    break   
            
            t2 = time.time()
            
            # Change the selected waveform of random channel every 5 seconds            
            if t2 - t1 >= 5:            
                t1 = time.time()
                irnd = random.randrange(num_channels)
                wav_indexes = [ (ind+irnd) % num_channels for ind in range(num_channels) ]
                ipacket = 0
                
                
            ipacket = (ipacket + 1) % num_packets

    except Exception as ex:
        print(ex)

    # At the end send ':TRACe:STReaming OFF' to stop streaming mode
    rc = inst.send_scpi_cmd(':TRAC:STR OFF')
    # Just for debug ..
    resp = inst.send_scpi_query(':TRAC:STR?')
    print('Streaming state: ' + resp)


## Main Part

In [ ]:


def main_func():
    global inst
    global keep_looping
    global lib_dir_path_
    global sampMode
    global sclk
    global slotIds
    
    keep_looping= True
    print('Start of main function')
    
    # Create a single instance of instruments-administrator.
    # Using the 'with' statement makes sure it will be closed at the end
    # Otherwise the user should close the administrator at the end with admin.close_inst_admin()

    
    admin = TepAdmin(lib_dir_path= lib_dir_path_)

    # Open a single-slot instrument

    # Please select appropriate slot-id
    #inst = admin.open_instrument(slotId)

    inst = admin.open_multi_slots_instrument(slotIds)

    resp = inst.send_scpi_query('*IDN?')
    print('Connected to: ' + resp)

    # Optionaly set paranoia-level (by default it is 1).
    # The supported values are: low (0), normal (1), or debug (2).
    inst.default_paranoia_level = 2

    # Reset the instrument
    rc = inst.send_scpi_cmd('*CLS; *RST')

    # Get the model name
    resp = inst.send_scpi_query(":SYST:iNF:MODel?")
    print("Model: " + resp)

    dataType = None
    # Infer the natural DAC waveform format
    if 'P908' in resp:
        dac_mode = 8
        dataType = np.uint8
    else:
        dac_mode = 16
        dataType = np.uint16
    print("DAC waveform format: {0} bits-per-point".format(dac_mode))

    # Get packet size:
    packet_size = inst.get_stream_packet_size()
    print( "packet size: {}".format(packet_size))
    
    print('Start streaming waves ..')

    num_packets = 16
    #num_cycles = 4

    # Start the streaming mode (in channel 1):
    resp = start_streaming_mode(inst, sclk, channels)

    # Push streaming-data while keep_looping is True
    push_streaming_data(inst, channels, fileNames, num_packets, packet_size, dataType)

    # Close instrument:
    #print('Closing instrument ..')    
    #inst.close_instrument()

    print('End of main function') 

# Run the main-function in thread
main_thread = Thread(target = main_func)

stop_streaming_button = widgets.Button(description='Stop Streaming')

def on_stop_streaming_button_clicked(arg):
    global inst
    global keep_looping
    global main_thread
    print('stop-streaming button clicked')
    keep_looping = False
    time.sleep(0.2)
    #if inst is not None:
    #    inst.send_scpi_cmd(':TRAC:STR OFF')
    #    inst.close_instrument()
    main_thread.join(5.0)

stop_streaming_button.on_click(on_stop_streaming_button_clicked)


# Start the main thread
main_thread.start()

# Display the button

display(stop_streaming_button)


    
    

## Stop Streaming

In [ ]:
#To Stop Streaming - run this function
on_stop_streaming_button_clicked(None)

In [ ]:
inst.close_instrument()